In [1]:
from __future__ import division
import osmgraph
import geog
import networkx as nx
import numpy as np
import geojsonio
import json
import itertools
from astar import A_star 
from time import time
import itertools
from random import shuffle, choice
from utils import *
from xml_creation import *

In [2]:
filename = '/Users/grigoriipogorelov/Desktop/melbourne.osm'
# filename = '/Users/grigoriipogorelov/Desktop/inno.osm'
g = osmgraph.parse_file(filename)

valid_nodes = {}
for n1, n2 in g.edges():
    valid_nodes[n1] = 0
    valid_nodes[n2] = 0
    c1, c2 = osmgraph.tools.coordinates(g, (n1, n2))
    g[n1][n2]['length'] = geog.distance(c1, c2)

a_star = A_star(g, valid_nodes)

### Experiment 1 (Table 1)

In [5]:
num_paths = [30, 40, 50]

for pairs in num_paths:
    valid_nodes_list = valid_nodes.keys()
    shuffle(valid_nodes_list)
    sources = valid_nodes_list[0:pairs]
    destinations = valid_nodes_list[pairs:2*pairs]
    
    
    ks = [1.5, 2, 3, 4, 5]
    range_ = 10
    repeats = 1
    Sd = []
    k_Sr = []
    stop_flag = False

    start = time()
    for k in ks:
        k_max = k
        step = (k_max-1)/range_
        k_range = list(np.arange(1, k_max, step))
        k_range.append(k_max)
        
        Sr = []
        for src, dst in itertools.izip(sources, destinations):
            try:
                temp = 0
                for i in range(repeats):
                    a_star_path, _ = a_star.a_star_search(src,dst, k_range)
                    temp += dist(g, a_star_path)
                temp = int(temp / repeats)
                Sr.append(temp)
                
                if not stop_flag:
                    shortest_path = nx.shortest_path(g, src, dst, 'length')
                    dist_ = dist(g, shortest_path)
                    Sd.append(dist_)
                
            except (KeyError, nx.NetworkXNoPath):
                continue
        
        k_Sr.append(Sr)
        stop_flag = True
    
    
    end = time()
    print
    print('number of paths %d' % pairs)
    print('time spent %.1f min' % ((end-start)/60))

    ACCs = []
    RUIs = []

    for k in k_Sr:
        acc = 0
        for sd, sr in itertools.izip(Sd, k):
            acc += ACC(sd, sr)
        ACCs.append(round(acc / len(Sd), 2))
        RUIs.append(round(RUI(Sd, k), 2))
    print('k ',ks)
    print('ACC ',ACCs)
    print('RUI ',RUIs)

# print Sd
# print
# for k in k_Sr:
#     print k


number of paths 42
time spent 0.2 min
('k ', [1.5, 2, 3, 4, 5])
('ACC ', [0.99, 0.98, 0.94, 0.94, 0.94])
('RUI ', [0.01, 0.02, 0.07, 0.06, 0.08])

number of paths 45
time spent 0.2 min
('k ', [1.5, 2, 3, 4, 5])
('ACC ', [0.99, 0.98, 0.96, 0.95, 0.93])
('RUI ', [0.01, 0.02, 0.04, 0.07, 0.08])

number of paths 47
time spent 0.4 min
('k ', [1.5, 2, 3, 4, 5])
('ACC ', [1.0, 0.98, 0.96, 0.93, 0.93])
('RUI ', [0.0, 0.03, 0.05, 0.08, 0.09])


### Experiment 2

In [3]:
num_cars = [50,100,200,500,1000,1500,2000]

sources = [370708433]
destinations = [1928298965]

shortest_paths, a_paths = [], []
for src, dst in itertools.izip(sources, destinations):
    path = nx.shortest_path(g, src, dst, 'length')
    shortest_paths.append(path)
#     coords = osmgraph.tools.coordinates(g, path)
#     paths.append({'type': 'LineString', 'coordinates': coords})
k = 2
range_ = 10
k_max = k
step = (k_max - 1) / range_
k_range = list(np.arange(1, k_max, step))
k_range.append(k_max)

variety = 6
a_star = A_star(g, valid_nodes)
for src, dst in itertools.izip(sources, destinations):
    for i in range(variety):
        a_star_path, _ = a_star.a_star_search(src, dst, k_range)
        a_paths.append(a_star_path)
#         coords = osmgraph.tools.coordinates(g, a_star_path)
#         paths.append({'type': 'LineString', 'coordinates': coords})


for n_cars in num_cars:
    all_xmls(g, shortest_paths, 7, n_cars, 'maps/melbourne_sh/melbourne_sh_' + str(n_cars))
    all_xmls(g, a_paths, 7, n_cars, 'maps/melbourne_a/melbourne_a_' + str(n_cars))

<?xml version="1.0" ?>
<routes>
	<vType accel="2.3" decel="4.1" id="Car0" length="14.3" maxSpeed="96" sigma="0.0"/>
	<vType accel="2.4" decel="7.5" id="Car1" length="10.0" maxSpeed="115" sigma="0.0"/>
	<vType accel="1.3" decel="7.3" id="Car2" length="7.1" maxSpeed="116" sigma="0.0"/>
	<vType accel="2.6" decel="2.7" id="Car3" length="16.2" maxSpeed="100" sigma="0.0"/>
	<vType accel="2.3" decel="2.9" id="Car4" length="6.8" maxSpeed="64" sigma="0.0"/>
	<vType accel="2.1" decel="4.6" id="Car5" length="14.7" maxSpeed="92" sigma="0.0"/>
	<vType accel="1.4" decel="2.5" id="Car6" length="16.7" maxSpeed="99" sigma="0.0"/>
	<route edges="370708433_2612216102 2612216102_1833869065 1833869065_320852621 320852621_370708381 370708381_370708343 370708343_370708264 370708264_370708114 370708114_320852504 320852504_250291418 250291418_253546192 253546192_2100795462 2100795462_250291504 250291504_3855646545 3855646545_3359255438 3359255438_36022333 36022333_36022334 36022334_253546237 253546237_36022335

In [69]:
files = ['/Users/grigoriipogorelov/Desktop/experiments/fig4/fig4_k=5/map.geojson']

for file in files:
    with open(file) as map_file:
        map_ = json.load(map_file)
geojsonio.display(json.dumps(map_))


# figs = []
# for file in files:
#     with open(file) as map_file:
#         if len(figs) == 0:
#             figs = json.load(map_file)
#         else:
#             figs.append(json.load(map_file)[0])

u'http://geojson.io/#data=data:application/json,%5B%7B%22type%22%3A%20%22LineString%22%2C%20%22coordinates%22%3A%20%5B%5B145.0209243%2C%20-37.782897%5D%2C%20%5B145.0209417%2C%20-37.7827142%5D%2C%20%5B145.0203705%2C%20-37.7828204%5D%2C%20%5B145.0197611%2C%20-37.782929%5D%2C%20%5B145.019505%2C%20-37.7829781%5D%2C%20%5B145.0191516%2C%20-37.7830204%5D%2C%20%5B145.0185843%2C%20-37.7831233%5D%2C%20%5B145.018267%2C%20-37.783193%5D%2C%20%5B145.0177583%2C%20-37.7832897%5D%2C%20%5B145.0174182%2C%20-37.7833483%5D%2C%20%5B145.0174044%2C%20-37.7834537%5D%2C%20%5B145.0171343%2C%20-37.7847412%5D%2C%20%5B145.0159432%2C%20-37.7845033%5D%2C%20%5B145.0143352%2C%20-37.7844044%5D%2C%20%5B145.0142645%2C%20-37.7843878%5D%2C%20%5B145.014118%2C%20-37.7844959%5D%2C%20%5B145.0140014%2C%20-37.7845105%5D%2C%20%5B145.0138874%2C%20-37.7845247%5D%2C%20%5B145.0136597%2C%20-37.7845577%5D%2C%20%5B145.0134349%2C%20-37.7845731%5D%2C%20%5B145.0133197%2C%20-37.7845711%5D%2C%20%5B145.01314%2C%20-37.7845948%5D%2C%20%5B145.012

In [62]:
for f in figs:
    print(type(f))

<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
